The purpose of this notebook is to put into practice what I have learned in the first lesson of Optimization for large scale data for my master in Big Data Analytics. The objective is to maximize the revenue for Google when 3 advertisers want to appear in the search results of 3 specific queries. The advertisers say how much they are willing to pay for each time their ad appears in the search results. Advertisers also have a budget limit, which is taken as a constraint. Another constraint is the maximum number of requests per query, estimated internally by Google. To optimize the number of appearances of each advertiser I use Linear Optimization. The library that I use is pyomo and the solver is glpk

## Table

  Revenue (€/user)     |   Q1   |   Q2   |   Q3   |   Q4   | Budget |
  ---------------------|--------|--------|--------|--------|--------|
  Company A            |   1    |  0.75  |   5    |   2    |  200   |
  Company B            |  0.5   |  0.5   |   2    |   3    |  150   |
  Company C            |  0.5   |   3    |   1    |  0.5   |  180   |
  Company D            |   2    |   1    |   3    |   1    |  250   |
  Query volume         |  150   |   90   |   80   |  120   |        |


## Sets 

\begin{align*}
    &AD = [A, B, C, D]\\
    &Q = [Q1, Q2, Q3, Q4]
\end{align*}

## In general formulation

\begin{align*}
\underset{x_{aq}}{\max} & \quad \sum_{q \in Q}\sum_{a \in AD}c_{a q}x_{a q}\\
\text{s.t.:}&\\
  &\sum_{q \in Q} c_{a q}\, x_{a q} \le B_{a} \quad \forall a \in AD\\
  &\sum_{a \in AD} x_{a q} \le d_{q} \quad \forall q \in Q
\end{align*}


Where
\begin{align*}
  &x = [X_{A1}, X_{A2}, X_{A3},X_{A4}, X_{B1}, X_{B2}, X_{B3}, X_{B4}, X_{C1}, X_{C2}, X_{C3}, X_{C4}, X_{D1}, X_{D2}, X_{D3}, X_{D4}]\\
  &B = [200, 150, 180, 250]\\
  &d = [150, 90, 80, 120]\\
  &c =
\begin{bmatrix}
1 & 0.75 & 5 & 2\\
0.5 & 0.5 & 2 & 3\\
0.5 & 3 & 1 & 0.5\\
2 & 1 & 3 & 1
\end{bmatrix}
\end{align*}


In [8]:
import numpy as np
import pyomo.environ as pyo
from pyomo.core.base.PyomoModel import ConcreteModel

opt = pyo.SolverFactory('glpk')
model = pyo.ConcreteModel(name = 'ads revenue')

#data
AD = ["A", "B", "C", "D"]
Q  = ["Q1", "Q2", "Q3", "Q4"]
c = {
    ("A","Q1"): 1,   ("A","Q2"): 0.75, ("A","Q3"): 5, ("A", "Q4"): 2,
    ("B","Q1"): 0.5, ("B","Q2"): 0.5,  ("B","Q3"): 2, ("B", "Q4"): 3,
    ("C","Q1"): 0.5, ("C","Q2"): 3,    ("C","Q3"): 1, ("C", "Q4"): 0.5,
    ("D","Q1"): 2,   ("D","Q2"): 1,    ("D","Q3"): 3, ("D", "Q4"): 1,
}
B = {"A": 200, "B": 150, "C": 180, "D": 250}
d = {"Q1": 150, "Q2": 90, "Q3": 80, "Q4": 120}

#sets
model.AD = pyo.Set(initialize = AD) #factories
model.Q = pyo.Set(initialize = Q) #markets

#params
model.c = pyo.Param(model.AD, model.Q, initialize=c)
model.B = pyo.Param(model.AD, initialize=B)
model.d = pyo.Param(model.Q, initialize=d)

#variables
model.x = pyo.Var(model.AD, model.Q, domain=pyo.NonNegativeIntegers)

#define the objective function
def obj_def(model):
    return sum(model.c[a,q] * model.x[a,q] for a in model.AD for q in model.Q)
model.OF = pyo.Objective(rule=obj_def, sense=pyo.maximize)

#define constraint for budget
def bud_const(model, a):
    return sum(model.c[a,q] * model.x[a,q] for q in model.Q) <= model.B[a]
model.bud = pyo.Constraint(model.AD, rule=bud_const)

#define constraint for volume of requests
def vol_const(model, q):
    return sum(model.x[a,q] for a in model.AD) <= model.d[q]
model.vol = pyo.Constraint(model.Q, rule=vol_const)

#results
result = opt.solve(model)
print('Status:', result.solver.status)
print('Condition:', result.solver.termination_condition)
print('Maximum revenue:', model.OF(), '€')
for a in model.AD:
    for q in model.Q:
        print(f'({a},{q}) --> shown {pyo.value(model.x[a, q])} times. Revenue of {pyo.value(model.x[a, q])*c[a,q]}€')

Status: ok
Condition: optimal
Maximum revenue: 780.0 €
(A,Q1) --> shown 0.0 times. Revenue of 0.0€
(A,Q2) --> shown 0.0 times. Revenue of 0.0€
(A,Q3) --> shown 40.0 times. Revenue of 200.0€
(A,Q4) --> shown 0.0 times. Revenue of 0.0€
(B,Q1) --> shown 0.0 times. Revenue of 0.0€
(B,Q2) --> shown 0.0 times. Revenue of 0.0€
(B,Q3) --> shown 0.0 times. Revenue of 0.0€
(B,Q4) --> shown 50.0 times. Revenue of 150.0€
(C,Q1) --> shown 0.0 times. Revenue of 0.0€
(C,Q2) --> shown 60.0 times. Revenue of 180.0€
(C,Q3) --> shown 0.0 times. Revenue of 0.0€
(C,Q4) --> shown 0.0 times. Revenue of 0.0€
(D,Q1) --> shown 65.0 times. Revenue of 130.0€
(D,Q2) --> shown 0.0 times. Revenue of 0.0€
(D,Q3) --> shown 40.0 times. Revenue of 120.0€
(D,Q4) --> shown 0.0 times. Revenue of 0.0€


## Results Table

  number of ads shown  |   Q1   |   Q2   |   Q3   |   Q4   |
  ---------------------|--------|--------|--------|--------|
  Company A            |   0    |   0    |   40   |   0    |
  Company B            |   0    |   0    |   0    |   50   |
  Company C            |   0    |   60   |   0    |   0    |
  Company D            |   65   |   0    |   40   |   0    |
  


In [9]:
model.display()

Model ads revenue

  Variables:
    x : Size=16, Index=AD*Q
        Key         : Lower : Value : Upper : Fixed : Stale : Domain
        ('A', 'Q1') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
        ('A', 'Q2') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
        ('A', 'Q3') :     0 :  40.0 :  None : False : False : NonNegativeIntegers
        ('A', 'Q4') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
        ('B', 'Q1') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
        ('B', 'Q2') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
        ('B', 'Q3') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
        ('B', 'Q4') :     0 :  50.0 :  None : False : False : NonNegativeIntegers
        ('C', 'Q1') :     0 :   0.0 :  None : False : False : NonNegativeIntegers
        ('C', 'Q2') :     0 :  60.0 :  None : False : False : NonNegativeIntegers
        ('C', 'Q3') :     0 :   0.0 :  None : False